In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import model_selection, preprocessing

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")
fx = pd.read_excel('./BAD_ADDRESS_FIX.xlsx').drop_duplicates('id').set_index('id')
train_df.update(fx)
test_df.update(fx)

In [4]:
train_df["datetime"] = pd.to_datetime(train_df["timestamp"])
train_df["year"] = train_df["datetime"].dt.year
train_df["month"] = train_df["datetime"].dt.month
train_df["day"] = train_df["datetime"].dt.day

In [5]:
temp = train_df['build_year'].copy()
temp[train_df["build_year"] > 1000] = train_df["year"][train_df["build_year"] > 1000] - train_df["build_year"][train_df["build_year"] > 1000]
train_df["build_year"] = temp

In [6]:
pt = pd.get_dummies(train_df["product_type"])

In [7]:
train_df = pd.concat([train_df, pt],axis=1)

In [8]:
test_df["datetime"] = pd.to_datetime(test_df["timestamp"])
test_df["year"] = test_df["datetime"].dt.year
test_df["month"] = test_df["datetime"].dt.month
test_df["day"] = test_df["datetime"].dt.day
temp2 = test_df['build_year'].copy()
temp2[test_df["build_year"] > 1000] = test_df["year"][test_df["build_year"] > 1000] - test_df["build_year"][test_df["build_year"] > 1000]
test_df["build_year"] = temp2

In [9]:
pt = pd.get_dummies(test_df["product_type"])
test_df = pd.concat([test_df, pt],axis=1)

In [10]:
train_df = train_df.replace({"no": 0, "yes": 1})
test_df = test_df.replace({"no": 0, "yes": 1})

In [11]:
inplace1 = (train_df["full_sq"] < train_df["life_sq"]) & (train_df["full_sq"] < 10)
temp3 = train_df["full_sq"].copy()
temp3[inplace1] = train_df["life_sq"][inplace1]
train_df["full_sq"] = temp3

In [12]:
inplace2 = train_df["life_sq"] < 2
temp4 = train_df["life_sq"].copy()
temp4[inplace2] = train_df["full_sq"][inplace2]
train_df["life_sq"] = temp4

In [13]:
inplace3 = (train_df["full_sq"] < train_df["life_sq"]) & (train_df["life_sq"] > 10)
temp5 = train_df["life_sq"].copy()
temp5[inplace3] = train_df["full_sq"][inplace3]
train_df["life_sq"] = temp5

In [14]:
inplace1 = (test_df["full_sq"] < test_df["life_sq"]) & (test_df["full_sq"] < 10)
temp3 = test_df["full_sq"].copy()
temp3[inplace1] = test_df["life_sq"][inplace1]
test_df["full_sq"] = temp3
inplace2 = test_df["life_sq"] < 2
temp4 = test_df["life_sq"].copy()
temp4[inplace2] = test_df["full_sq"][inplace2]
test_df["life_sq"] = temp4
inplace3 = (test_df["full_sq"] < test_df["life_sq"]) & (test_df["life_sq"] > 10)
temp5 = test_df["life_sq"].copy()
temp5[inplace3] = test_df["full_sq"][inplace3]
test_df["life_sq"] = temp5

In [15]:
ma = pd.read_csv("./macro.csv")

In [16]:
ma_chosen = ma[["timestamp","gdp_quart", "gdp_quart_growth", "cpi", "ppi", "oil_urals"]]

In [17]:
ma_chosen["datetime"] = pd.to_datetime(ma["timestamp"])

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
ma_chosen.drop(['timestamp'], axis=1, inplace=True)

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [19]:
train_df = pd.merge(train_df, ma_chosen, how='left', on='datetime')
test_df = pd.merge(test_df, ma_chosen, how='left', on='datetime')

In [20]:
train_df["build_age"] = train_df["year"] - train_df["build_year"]
test_df["build_age"] = test_df["year"] - test_df["build_year"]
train_df["life_full_ratio"] = train_df["life_sq"] / train_df["full_sq"]
test_df["life_full_ratio"] = test_df["life_sq"] / test_df["full_sq"]
train_df["floor_position"] = train_df["floor"] / train_df["max_floor"]
test_df["floor_position"] = test_df["floor"] / test_df["max_floor"]

In [21]:
y_train = train_df['price_doc'].values
id_test = test_df['id']
train_df.drop(['timestamp'], axis=1, inplace=True)
test_df.drop(['timestamp'], axis=1, inplace=True)
train_df.drop(['datetime'], axis=1, inplace=True)
test_df.drop(['datetime'], axis=1, inplace=True)

In [22]:
train_eco = pd.get_dummies(train_df["ecology"])
train_df = pd.concat([train_df, train_eco],axis=1)

In [23]:
test_eco = pd.get_dummies(test_df["ecology"])
test_df = pd.concat([test_df, test_eco],axis=1)

In [24]:
train_df.drop(['product_type'], axis=1, inplace=True)
test_df.drop(['product_type'], axis=1, inplace=True)
train_df.drop(['ecology'], axis=1, inplace=True)
test_df.drop(['ecology'], axis=1, inplace=True)

In [25]:
train_df.drop("id", axis=1, inplace=True)
train_df.drop("price_doc", axis=1, inplace=True)
train_df.drop("sub_area", axis=1, inplace=True)
test_df.drop("id", axis=1, inplace=True)
test_df.drop("sub_area", axis=1, inplace=True)

In [25]:
train_df.shape

(30471, 304)

In [26]:
test_df.shape

(7662, 304)

In [26]:
dtrain = xgb.DMatrix(train_df.values, y_train)

In [27]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 4,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'min_child_weight':1,
    'silent': 1,
    }
    

In [28]:
xgb.cv(xgb_params, dtrain, nfold=5, num_boost_round=1000, early_stopping_rounds=20,
    verbose_eval=20, show_stdv=False)

[0]	train-rmse:8.23111e+06	test-rmse:8.23282e+06
[20]	train-rmse:4.10055e+06	test-rmse:4.18413e+06
[40]	train-rmse:2.97671e+06	test-rmse:3.15725e+06
[60]	train-rmse:2.6602e+06	test-rmse:2.91738e+06
[80]	train-rmse:2.52799e+06	test-rmse:2.83891e+06
[100]	train-rmse:2.45234e+06	test-rmse:2.80092e+06
[120]	train-rmse:2.39082e+06	test-rmse:2.7736e+06
[140]	train-rmse:2.34251e+06	test-rmse:2.75629e+06
[160]	train-rmse:2.30262e+06	test-rmse:2.74126e+06
[180]	train-rmse:2.26647e+06	test-rmse:2.7301e+06
[200]	train-rmse:2.23555e+06	test-rmse:2.72172e+06
[220]	train-rmse:2.20588e+06	test-rmse:2.71128e+06
[240]	train-rmse:2.18119e+06	test-rmse:2.70518e+06
[260]	train-rmse:2.15756e+06	test-rmse:2.69999e+06
[280]	train-rmse:2.1352e+06	test-rmse:2.69207e+06
[300]	train-rmse:2.11561e+06	test-rmse:2.68755e+06
[320]	train-rmse:2.09446e+06	test-rmse:2.68368e+06
[340]	train-rmse:2.07583e+06	test-rmse:2.67966e+06
[360]	train-rmse:2.05713e+06	test-rmse:2.67662e+06
[380]	train-rmse:2.03957e+06	test-rmse:2.

,test-rmse-mean,test-rmse-std,train-rmse-mean,train-rmse-std
0,8232822.00,140230.541212,8231106.100,34308.904507
1,7896753.50,135214.496032,7892679.400,41998.278933
2,7579428.20,136042.132458,7573895.400,46208.130099
3,7278498.30,136247.488951,7269881.800,47738.738338
4,6987265.60,137527.604000,6976358.800,45976.487964
5,6714799.40,140280.112602,6700219.600,44550.012954
6,6459857.20,147453.926840,6441043.900,40276.323976
7,6220638.20,149661.019330,6199446.400,41443.260776
8,5995006.00,156446.371852,5968811.200,35995.074753
9,5786114.00,156378.529069,5756617.700,39665.725742


In [29]:
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= 497)

In [30]:
dtest = xgb.DMatrix(test_df.values)

In [31]:
y_predict = model.predict(dtest)
output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})

In [32]:
output.mean()

id           3.430450e+04
price_doc    7.875476e+06
dtype: float64

In [35]:
output.mean()

id           3.430450e+04
price_doc    7.872189e+06
dtype: float64

In [33]:
output.to_csv('FinalFight3.csv', index=False)